1. Imports and Setup

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import random
import gc
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


2. Paths, Transforms, and Data Loading

In [2]:
# Data paths
train_dir = 'weather_dataset'
test_dir = 'test'

IMG_HEIGHT, IMG_WIDTH = 300, 300
BATCH_SIZE = 8
SEED = 42

# Image transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop((IMG_HEIGHT, IMG_WIDTH), scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(40),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


3. Custom Dataset and Dataloaders

In [3]:
class WeatherDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        if self.labels is not None:
            return img, self.labels[idx]
        else:
            return img, os.path.basename(img_path)

# Class mapping
classes = sorted(os.listdir(train_dir))
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

# Collect image paths and labels
all_images = []
all_labels = []
for cls in classes:
    imgs = glob(os.path.join(train_dir, cls, "*.jpg"))
    all_images.extend(imgs)
    all_labels.extend([class_to_idx[cls]] * len(imgs))

# Stratified K-Fold split
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(all_images, all_labels))

train_imgs = [all_images[i] for i in train_idx]
train_labels = [all_labels[i] for i in train_idx]
val_imgs = [all_images[i] for i in val_idx]
val_labels = [all_labels[i] for i in val_idx]

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

# Datasets
train_dataset = WeatherDataset(train_imgs, train_labels, transform=train_transforms)
val_dataset = WeatherDataset(val_imgs, val_labels, transform=val_transforms)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# Test loader
test_image_paths = sorted(glob(os.path.join(test_dir, "*/*.jpg")))
test_dataset = WeatherDataset(test_image_paths, transform=val_transforms)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)


4. Model Definition

In [4]:
# Load EfficientNetB3
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
model = efficientnet_b3(weights=EfficientNet_B3_Weights.DEFAULT)

# Freeze base layers
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, len(classes))
)
model = model.to(device)


Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to C:\Users\HomePC/.cache\torch\hub\checkpoints\efficientnet_b3_rwightman-b3899882.pth


100%|██████████| 47.2M/47.2M [00:07<00:00, 6.88MB/s]


5. Training set-up

In [5]:
# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss(label_smoothing=0.1, weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-6)

# Training control
EPOCHS = 20
patience = 5
best_val_acc = 0.0
counter = 0


6. Training Loop

In [7]:
for epoch in range(EPOCHS):
    model.train()
    train_loss, correct, total = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total

    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}: Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_weather_model.pth")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

    gc.collect()
    torch.cuda.empty_cache()


RuntimeError: DataLoader worker (pid(s) 38904, 11876) exited unexpectedly

7. Inference and Submission

In [17]:
# Load best model
model.load_state_dict(torch.load("best_weather_model.pth"))
model.eval()

# Predict
predictions = []
with torch.no_grad():
    for i, (images, _) in enumerate(test_loader, start=1):  # start numbering at 1
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.append((i, preds.item()))

# Map back to class names
idx_to_class = {v: k for k, v in class_to_idx.items()}
submission_df = pd.DataFrame({
    "id": [i for i, _ in predictions],  # numeric ids only
    "label": [idx_to_class[p] for _, p in predictions]
})

# Save
submission_df.to_csv("pytorchsubmission.csv", index=False)
submission_df.head()


,id,label
0,1,Cloudy
1,2,Cloudy
2,3,Cloudy
3,4,Cloudy
4,5,Shine
